# Simulation Analysis 

In [1]:
import symupy
import pandas as pd
import numpy as np 
import os
import sys
import re
from itertools import repeat, chain
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

try:
    pd.set_option('plotting.backend', 'hvplot')
except: 
    pass
print(f"Backend: {pd.options.plotting.backend}")

from symupy.api import Simulator,Simulation

print(f"Version of symupy: {symupy.__version__}")

packages = ['src', 'data/results/']

# Adding supplementary functions 
for pck in packages:
    print(f"Adding folder: {pck}")
    sys.path.append(os.path.join(os.getcwd(),f"../{pck}"))


Successful import of symupy
Backend: hvplot
Version of symupy: 0.2.0
Adding folder: src
Adding folder: data/results/


In [43]:
# Cases to read
output_dir = os.listdir('../data/results')
output_dir
output_dir = 'Cpt_5_MANUAL_90.0/'
load_dir = os.getcwd()+'/../data/results/'+ output_dir

In [44]:
filettt = load_dir +'ttt.csv'
ttt = pd.read_csv(filettt)
ttt.plot(title = 'Closed loop - Total Travel Time - Vanishing 30%',grid = True) # Vanishing 0%

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [45]:
filettd = load_dir+'ttd.csv'
ttd = pd.read_csv(filettd)
ttd.plot(title = 'Closed loop - Total Travel Distance - Vanishing 30%',grid = True) # Vanishing 0%

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [46]:
filespd = load_dir+'spd.csv'
spd = pd.read_csv(filespd)
spd.plot(title = 'Closed loop - Zone Speed - Vanishing 30%', grid = True) # Vanishing 0%

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [47]:
filectr = load_dir+'ctr.csv'
ctr = pd.read_csv(filectr)
ctr.plot(title = 'Open loop - Zone Control -  Vanishing', grid = True) # Vanishing 0%

:NdOverlay   [Variable]
   :Curve   [index]   (value)